In [41]:
#read data
import pandas as pd
import numpy as np
train_data = pd.read_csv("house-prices-advanced-regression-techniques/train.csv")
train_X, train_Y  = train_data.iloc[:,1:-1], train_data.iloc[:,-1:]      #not including ID. Keep data in order for cooresponding Y
test_data = pd.read_csv("house-prices-advanced-regression-techniques/test.csv").iloc[:,1:]
test_X, test_Y  = test_data, pd.read_csv("house-prices-advanced-regression-techniques/sample_submission.csv")['SalePrice']

print(train_X.columns)
print(train_Y.columns)
print(train_X.columns)
print(train_Y.columns)
print(f"train_X.shape[1] {train_X.shape[1]}, test_X.shape[1] {test_X.shape[1]}")
assert train_X.shape[1] == test_X.shape[1], print(f"train_X.shape[1] {train_X.shape[1]}, test_X.shape[1] {test_X.shape[1]}")
assert train_X.shape[0] == train_Y.shape[0] 
assert test_X.shape[0] == test_Y.shape[0] 


Index(['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'Alley',
       'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
       'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
       'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea',
       'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC',
       'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd',
       'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
       'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond',
       'PavedDrive', 'Wo

In [42]:
#clean data
print(f"train_X.shape[1] {train_X.shape[1]}, test_X.shape[1] {test_X.shape[1]}")

#first combine data, so that get_dummies will have all classes
all_data = pd.concat([train_X, test_X], axis=0)

all_data_dummy = pd.get_dummies(all_data)
all_data_dummy = all_data_dummy.fillna(0)

train_X = all_data_dummy.iloc[:train_X.shape[0]]
test_X =  all_data_dummy.iloc[train_X.shape[0]:]
#print(clear_data.columns)



assert train_X.shape[1] == test_X.shape[1], print(f"train_X.shape[1] {train_X.shape[1]}, test_X.shape[1] {test_X.shape[1]}")

train_X.to_csv("train_X.txt", sep="\t", index=False)
test_X.to_csv("test_X.txt", sep="\t", index=False)

train_X.shape[1] 79, test_X.shape[1] 79


In [43]:
def normalize(column):
    return (column - column.min()) / (column.max() - column.min())

train_X = train_X.astype(float)
test_X  = test_X.astype(float)

train_X = train_X.apply(normalize)
test_X  = test_X.apply(normalize)

train_X.to_csv("train_X.txt", sep="\t", index=False)
test_X.to_csv("test_X.txt", sep="\t", index=False)

In [44]:
#Linear Regression
#first do PCA
print(f"train_X.shape {train_X.shape}")

train_X.shape (1460, 287)
